<a href="https://colab.research.google.com/github/Neeshi14/pneumoniadisease/blob/main/pneumoniadiseaseclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pneumonia Disease Detection Model**

 These is the pneumonia disease classification model. In these model ,We are using the pretrained model of CNN and bulid the model using the pytorch framework .


In CNN there are many pretraied model but in these we are used the ResNet model.



Import the data using Kaggle Dataset

In [1]:
!mkdir -p ~/.kaggle
!mv /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download dataset (replace with actual dataset name)
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

# Extract ZIP file
import zipfile

zip_path = "/content/chest-xray-pneumonia.zip"
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


mv: cannot stat '/content/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
 99% 2.28G/2.29G [00:20<00:00, 254MB/s]
100% 2.29G/2.29G [00:20<00:00, 121MB/s]


Import the library

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import os
from PIL import Image

In [3]:
#Setup and Data Loading
train_dir = '/content/dataset/chest_xray/train'
test_dir = '/content/dataset/chest_xray/test'
val_dir = '/content/dataset/chest_xray/val'

In [4]:
# Image Parameters
img_width, img_height = 150, 150
batch_size = 32

In [5]:
# Data Transformations
train_transforms = transforms.Compose([
    transforms.Resize((img_width, img_height)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]), # ImageNet stats
])

test_transforms = transforms.Compose([
    transforms.Resize((img_width, img_height)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

In [6]:
# Custom Dataset Class
class PneumoniaDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.image_paths = []
        self.labels = []

        for label in self.classes:
            label_dir = os.path.join(root_dir, label)
            for image_file in os.listdir(label_dir):
                image_path = os.path.join(label_dir, image_file)
                self.image_paths.append(image_path)
                self.labels.append(0 if label == 'NORMAL' else 1)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('L')  # Open as grayscale ('L')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float32)

In [7]:
# Create DataLoaders
train_dataset = PneumoniaDataset(train_dir, transform=train_transforms)
test_dataset = PneumoniaDataset(test_dir, transform=test_transforms)
val_dataset = PneumoniaDataset(val_dir, transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [8]:
# 2. Load Pretrained ResNet50 (Modified for Grayscale)

model = torchvision.models.resnet50(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2) #Change first layer to accept single channel.

# Freeze pretrained layers
for param in model.parameters():
    param.requires_grad = False

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 126MB/s]


In [9]:
# Load VGG16 pre-trained model
#vgg16 = torchvision.models.vgg16(pretrained=True)

# Modify the classifier (final layers) for binary classification
#num_features = vgg16.classifier[6].in_features
#vgg16.classifier[6] = nn.Linear(num_features, 1)  # Output layer for binary classification


In [10]:
# 3. Add Custom Classification Layers

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 128),
    nn.ReLU(),
    nn.Linear(128, 1),
    nn.Sigmoid()
)

# Move model to GPU (if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1):

In [11]:
# 4. Compile and Train

criterion = nn.BCELoss() #BCS-> binaray cross entropy loss
optimizer = optim.Adam(model.fc.parameters(), lr=0.001) # for VGG16 vgg16.parameters()

num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

Epoch 1/5, Loss: 0.31661783949546285
Epoch 2/5, Loss: 0.2307733095542419
Epoch 3/5, Loss: 0.2055499112587765
Epoch 4/5, Loss: 0.20738393006606337
Epoch 5/5, Loss: 0.20064989671659617


In [14]:
# 5. Evaluate
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")

Test Accuracy: 80.92948717948718%
